In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [70]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")

dummies = pd.get_dummies(train_data['label'], prefix='label', drop_first=False)
train_data = train_data
train_data = train_data.drop('label', axis=1)
labels = dummies

#print(labels)

train_samples, train_labels =train_data, labels
test_samples = test_data

print('Training set', train_samples.shape, train_labels.shape)
print('    Test set', test_samples.shape)

image_size = 28
num_channels = 1
num_labels = 10

Training set (42000, 784) (42000, 10)
    Test set (28000, 784)


In [132]:
def get_chunk(samples, labels, chunkSize):
    '''
    Iterator/Generator: get a batch of data
    这个函数是一个迭代器/生成器，用于每一次只得到 chunkSize 这么多的数据
    用于 for loop， just like range() function
    '''
#     if len(samples) != len(labels):
#         raise Exception('Length of samples and labels must equal')
#     stepStart = 0	# initial step
#     i = 0
#     while stepStart < len(samples):
#         stepEnd = stepStart + chunkSize
#         if stepEnd < len(samples):
#             yield i, samples[stepStart:stepEnd], labels[stepStart:stepEnd]
#         i += 1
#         stepStart = stepEnd
    datas = pd.concat([labels, samples], axis=1)
    random_batch = datas.loc[np.random.randint(42000, size=100)]
    images = random_batch.iloc[:, 10:].values
    images = images.astype(np.float)
    # convert from [0:255] => [0.0:1.0]
    images = np.multiply(images, 1.0 / 255.0)
        
    labels = random_batch.iloc[:, 0:10].values
    return images, labels
    

class Network():
    def __init__(self, num_hidden, batch_size):
        '''
        @num_hidden: 隐藏层的节点数量
        ＠batch_size: 节省内存 所以分批处理数据 每一批的数据量
        '''
        self.batch_size = batch_size
        self.test_batch_size = 28000
        
        # hyper parameters (超参数)
        self.num_hidden = num_hidden
        
        # Graph Related (计算图谱相关)
        self.graph = tf.Graph()
        self.tf_train_samples = None
        self.tf_train_labels = None
        self.tf_test_samples = None
        self.tf_test_labels = None
        self.tf_test_prediction = None
        
        
    def define_graph(self):
        '''
        定义计算图谱
        '''
        with self.graph.as_default():
            # 这里只是定义图谱中的各种变量
            self.tf_train_samples = tf.placeholder(
                tf.float32, shape=(self.batch_size, 784)
            )
            self.tf_train_labels  = tf.placeholder(
                tf.float32, shape=(self.batch_size, num_labels)
            )
            self.tf_test_samples  = tf.placeholder(
                tf.float32, shape=(self.test_batch_size, 784)
            )
            
            # 全连接的第一层(layer 1)，注意这里并没有权重共享
            fc1_weights = tf.Variable(
                tf.truncated_normal([image_size * image_size, self.num_hidden], stddev=0.1)
            )
            # fc1_biases = tf.Variable(tf.zeros([self.num_hidden]))
            fc1_biases = tf.Variable(tf.constant(0.1, shape=[self.num_hidden]))
            
            # 全连接的第二层(layer 2)
            fc2_weights = tf.Variable(
                tf.truncated_normal([self.num_hidden, num_labels], stddev=0.1)
            )
            fc2_biases = tf.Variable(tf.constant(0.1, shape=[num_labels]))
            
            # 定义图谱的运算
            def model(data):
                # fc1
                #shape = data.get_shape().as_list()
                #print(data.get_shape(), shape)
                #reshape
                #reshape_data = tf.reshape(data, [shape[0], shape[1] * shape[2] * shape[3]])
                hidden = tf.nn.relu(tf.matmul(data, fc1_weights) + fc1_biases)
                
                # fc2
                return tf.matmul(hidden, fc2_weights) + fc2_biases
            
            # Training computation. (训练)
            logits = model(self.tf_train_samples)
            self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.tf_train_labels)
            )

            # Optimizer. (优化器)
            self.optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(self.loss)

            # Predictions for the training, validation, and test data. (预测数据)
            self.train_prediction = tf.nn.softmax(logits)
            self.test_prediction = tf.nn.softmax(model(self.tf_test_samples))
            
        
    def run(self):
        '''
        用到Session
        '''
        # 训练
        self.session = tf.Session(graph=self.graph)
        with self.session as session:
            tf.global_variables_initializer().run()
            print('开始训练')
            # batch 1000
            for i in range(1000):
                samples, labels = get_chunk(train_samples, train_labels, chunkSize=self.batch_size)
                _, l, predictions = session.run(
                    [self.optimizer, self.loss, self.train_prediction],
                    feed_dict={self.tf_train_samples: samples, self.tf_train_labels: labels}
                )
                #print(labels.shape)
                accuracy, _ = self.accuracy(predictions, labels)
                if i % 50 == 0:
                    print('Minibatch loss as step %d: %f' % (i, l))
                    print('Minibatch accuracy: %.lf%%' % accuracy)
            #test_predictions = session.run([self.test_prediction], feed_dict={self.tf_test_samples: test_samples})
            #print("test shape is {}".format(test_predictions))
    
    def accuracy(self, predictions, labels, need_confusion_matrix=False):
        '''
        计算预测的正确率与召回率
        @return: accuracy and confusionMatrix as a tuple
        '''
        #print(type(predictions.shape))
        _predictions = np.argmax(predictions, 1)
        #print(_predictions)
        #print(labels.shape)
        #labels = [tuple(x) for x in labels.to_records(index=False)]
        labels = tuple(map(tuple, labels))
        _labels = np.argmax(labels, 1)
        #print(_labels.shape)
        cm = confusion_matrix(_labels, _predictions) if need_confusion_matrix else None
        # == is overloaded for numpy array
        accuracy = (100.0 * np.sum(_predictions == _labels) / predictions.shape[0])
        return accuracy, cm

In [134]:
net = Network(num_hidden = 256, batch_size = 100)
print(type(net))
net.define_graph()
net.run()

<class '__main__.Network'>
开始训练
Minibatch loss as step 0: 2.726348
Minibatch accuracy: 1%
Minibatch loss as step 50: 1.840623
Minibatch accuracy: 45%
Minibatch loss as step 100: 1.340141
Minibatch accuracy: 69%
Minibatch loss as step 150: 1.280504
Minibatch accuracy: 69%
Minibatch loss as step 200: 0.866129
Minibatch accuracy: 83%
Minibatch loss as step 250: 0.825664
Minibatch accuracy: 79%
Minibatch loss as step 300: 0.743143
Minibatch accuracy: 84%
Minibatch loss as step 350: 0.632484
Minibatch accuracy: 84%
Minibatch loss as step 400: 0.681659
Minibatch accuracy: 82%
Minibatch loss as step 450: 0.732495
Minibatch accuracy: 77%
Minibatch loss as step 500: 0.670477
Minibatch accuracy: 85%
Minibatch loss as step 550: 0.639655
Minibatch accuracy: 82%
Minibatch loss as step 600: 0.563874
Minibatch accuracy: 85%
Minibatch loss as step 650: 0.536616
Minibatch accuracy: 85%
Minibatch loss as step 700: 0.490733
Minibatch accuracy: 84%
Minibatch loss as step 750: 0.659809
Minibatch accuracy: 